## AU DEEP LEARNING COURSE:
### A minimal training routing using pre-defined CNN models
This nodebook sets up a simple training routine. To keep it simple, it contains no validation routine, which you would normally incorporate as part of the training loop.

### We need some training-data.
The first part downloads some traing data, unfortunatly the download-script it is not very stable :-/

Do you already have some training data? Great! Then you can skip this part - You just have to make sure the images are in individual folders, where the folder name indicate the class i.e.

In [ ]:
# path for training data
traindir = 'Trainingdata'

In [ ]:
import requests
import re
import json
import time
import os

def createifnotexist(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def search(keywords, outputfolder='./', max_results=None):
    outdir = os.path.join(outputfolder,keywords)
    createifnotexist(outdir)
    n_results = 0

    url = 'https://duckduckgo.com/'
    params = {
    	'q': keywords
    }

    #   First make a request to above URL, and parse out the 'vqd'
    #   This is a special token, which should be used in the subsequent request
    res = requests.post(url, data=params)
    searchObj = re.search(r'vqd=([\d-]+)\&', res.text, re.M|re.I)

    if not searchObj:
        return -1

    headers = {
        'authority': 'duckduckgo.com',
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'sec-fetch-dest': 'empty',
        'x-requested-with': 'XMLHttpRequest',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'cors',
        'referer': 'https://duckduckgo.com/',
        'accept-language': 'en-US,en;q=0.9',
    }

    params = (
        ('l', 'us-en'),
        ('o', 'json'),
        ('q', keywords),
        ('vqd', searchObj.group(1)),
        ('f', ',,,'),
        ('p', '1'),
        ('v7exp', 'a'),
    )

    requestUrl = url + "i.js"

    while True:
        while True:
            try:
                res = requests.get(requestUrl, headers=headers, params=params)
                data = json.loads(res.text)
                break
            except ValueError as e:
                time.sleep(5)
                continue

        for x in data["results"]:
            #os.system('wget '+x['image']+' -P "'+keywords+'"')
            os.system('wget "'+x['image']+'" --no-verbose -P "'+outdir+'"')
            n_results+=1
            if max_results is not None:
                if n_results>=max_results: return

        if "next" not in data:
            exit(0)

        requestUrl = url + data["next"]

In [ ]:
if not os.path.exists(os.path.join(traindir, "Dog")):
    search("Dog", outputfolder=traindir, max_results=100)
if not os.path.exists(os.path.join(traindir, "Cat")):
    search("Cat", outputfolder=traindir, max_results=100)

2022-09-28 17:58:07 URL:https://www.thesprucepets.com/thmb/AlsaZ1vMyplHqJkbe-iKeyzJwRY=/4368x2912/filters:fill(auto,1)/burmese-cat-love-85151671-5c87d723c9e77c0001a3e5ae.jpg [1287189/1287189] -> "Trainingdata/Cat/burmese-cat-love-85151671-5c87d723c9e77c0001a3e5ae.jpg" [1]
2022-09-28 17:58:07 URL:https://www.readersdigest.ca/wp-content/uploads/2019/11/most-affectionate-cat-breeds-russian-blue.jpg [163154/163154] -> "Trainingdata/Cat/most-affectionate-cat-breeds-russian-blue.jpg" [1]
2022-09-28 17:58:08 URL:http://images6.fanpop.com/image/photos/34300000/Kitten-cats-34352405-1600-1200.jpg [383197/383197] -> "Trainingdata/Cat/Kitten-cats-34352405-1600-1200.jpg" [1]
2022-09-28 17:58:08 URL:https://cdn.vox-cdn.com/thumbor/3GvnfOvGZB3paInd57sdlY1Hvu8=/0x0:1006x1014/1200x0/filters:focal(0x0:1006x1014):no_upscale()/cdn.vox-cdn.com/uploads/chorus_asset/file/22438217/Screen_Shot_2021_04_12_at_8.16.17_AM.png [195574/195574] -> "Trainingdata/Cat/Screen_Shot_2021_04_12_at_8.16.17_AM.png" [1]
2022-0



---



## Now for the training part

In [ ]:
#First som imports
import torch
import imageio
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

In [ ]:
# Use the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Set up augmentation and scaling

In [ ]:
input_shape=(224,224)
transformTrain = transforms.Compose(
    [lambda x: x[:,:,:3], # remove alpha channel
     transforms.ToPILImage(),
     transforms.Resize(input_shape),
     torchvision.transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transformTest = transforms.Compose(
    [lambda x: x[:,:,:3], # remove alpha channel
     transforms.ToPILImage(),
     transforms.Resize(input_shape),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


trainset = torchvision.datasets.folder.DatasetFolder(root=traindir, loader=imageio.imread, extensions=['jpg','png','tiff','jpeg'], transform=transformTrain)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=0)

Define network architecture

In [ ]:
num_classes = len(trainset.classes)

# Load existing model (Resnet inetic out with the majority of its major new features. There’s a new ‘Quick Settings’ that makes managing everyday system tasks easier; a resize-friendly Settings app; and ample new features in the Nautilus file manager, including restyled ‘properties’ dialogs and enhanced list view.18)
# Check-out this list: https://pytorch.org/vision/stable/models.html#classification
net = torchvision.models.resnet18(pretrained=True)

# change last layer to reflect the number of classes of our dataset.
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, num_classes)

input_size = 224


# Move network to GPU if available
net.to(device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Set-up training
criterion = nn.CrossEntropyLoss()

#optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9, nesterov=True)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
#optimizer = torch.optim.Adagrad(net.parameters(), lr=0.01, lr_decay=0, weight_decay=0.0005)


#Learning rate updater
#lambda1 = lambda epoch: epoch // 30#New, not tested
#lambda2 = lambda epoch: 0.97 ** epoch#New, not tested
#scheduler = torch.optim.lr_scheduler.LambdaLR(self.optimizer, lr_lambda=lambda2)#New, not tested
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, verbose=True)

## Training loop!

In [ ]:
NUM_EPOCHS = 100
itercounter=0
print_interval=10 #
for epoch in range(NUM_EPOCHS):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        net.train() # enable dropout and BatchNorm
        itercounter+=1

        inputs, labels = data
        # Move to GPU (or CPU)
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if (itercounter) % print_interval == print_interval-1:    # print every x mini-batches
            print('[epoch: %d, iter: %5d] running loss: %.6f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
net.eval() # disable dropout and BatchNorm

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2649113790>
Traceback (most recent call last):
  File "/home/mads/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/home/mads/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
Exception ignored in: Exception ignored in: Exception ignored in: AssertionError<function _MultiProcessingDataLoaderIter.__del__ at 0x7f2649113790><function _MultiProcessingDataLoaderIter.__del__ at 0x7f2649113790>

Traceback (most recent call last):
  File "/home/mads/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2649113790>can only test 

[epoch: 1, iter:     9] loss: 0.001091
[epoch: 1, iter:    19] loss: 0.001077
[epoch: 1, iter:    29] loss: 0.001302
[epoch: 2, iter:     3] loss: 0.000400
[epoch: 2, iter:    13] loss: 0.003485
[epoch: 2, iter:    23] loss: 0.001241
[epoch: 2, iter:    33] loss: 0.001755
[epoch: 3, iter:     7] loss: 0.000915
[epoch: 3, iter:    17] loss: 0.001085
[epoch: 3, iter:    27] loss: 0.000908
[epoch: 4, iter:     1] loss: 0.000009
[epoch: 4, iter:    11] loss: 0.000627
[epoch: 4, iter:    21] loss: 0.001554
[epoch: 4, iter:    31] loss: 0.000754
[epoch: 5, iter:     5] loss: 0.000471
[epoch: 5, iter:    15] loss: 0.001443
[epoch: 5, iter:    25] loss: 0.000807
[epoch: 5, iter:    35] loss: 0.001297
[epoch: 6, iter:     9] loss: 0.000745
[epoch: 6, iter:    19] loss: 0.000732
[epoch: 6, iter:    29] loss: 0.000920
[epoch: 7, iter:     3] loss: 0.000270
[epoch: 7, iter:    13] loss: 0.001166
[epoch: 7, iter:    23] loss: 0.000688
[epoch: 7, iter:    33] loss: 0.001766
[epoch: 8, iter:     7] l

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Let's evaluate the model on some training data

In [ ]:
# Test the model
import imageio
import numpy as np
image = imageio.imread('Testdata/cat_image.jpg')

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

imagetransformed = transformTest(image)
if len(imagetransformed.shape)==3:
    imagetransformed = imagetransformed.unsqueeze(0)

imagetransformed = imagetransformed.to(device)

outputs = net(imagetransformed)
#_, predicted = torch.max(outputs.data, 1)

output_prob = outputs.data.tolist()[0]

softmaxOutput = True
if softmaxOutput:
    output_prob = softmax(output_prob)
    #print(output_prob)

[0.68716772 0.31283228]


In [ ]:
print('Prediction:', trainset.classes[np.argmax(output_prob)])

Prediction: Cat
